In [14]:
#
#todo New class that inherits from AbstractExtraction and constructs with the information from the secret store an
# exasol.ai.text.extraction.Extraction object and calls its run method

from exasol.ai.text.extraction import *
from exasol.ai.text.extraction.extraction import Extraction
from exasol.ai.text.extraction import AbstractExtraction
from exasol.nb_connector.connections import open_pyexasol_connection
from exasol.ai.text.extraction.abstract_extraction import Output
from exasol.nb_connector.secret_store import Secrets #todo is this installed here?
from exasol.nb_connector.ai_lab_config import AILabConfig as CKey
from exasol.nb_connector.language_container_activation import get_activation_sql


#text_column="TEXT_COLUMN"
#key_column="KEY_COLUMN"
#table="MY_TABLE"
#table="CUSTOMER_SUPPORT_TICKETS"
text_column="TICKET_DESCRIPTION"
key_column="TICKET_ID"
view="MY_VIEW"
LABELS=["urgent", "not urgent"]#todo which of these are in cof and which are input?
topics=["hardware issue", "software issue"]

#class AbstractExtraction(BaseModel):
    # Ignore cython function in raw pydantic class, see https://github.com/pydantic/pydantic/issues/6670
#    model_config = {'ignored_types': (CyFunctionDetector,), 'frozen': True}

#    extractor: Annotated[Extractor, AfterValidator(validate_source)]
#    output: Output
##############################

#    id: int
 #   name: str = 'Jane Doe'

 #   model_config = ConfigDict(str_max_length=10) 

class ExtractionWrapper(Extraction, AbstractExtraction):#todo do we nedd the second?
    def __init__(self, conf: Secrets):# todo get info from conf
        schema=conf.db_schema
        super().__init__(
            extractor=PipelineExtractor( #todo or are these input for the wrapper? probably
                steps=[
                    SourceTableExtractor(sources=[#todo or does all this happen in the run()?
                        SchemaSource(db_schema=NameSelector(pattern=schema),
                             tables=[
                                 TableSource(table=NameSelector(pattern=view),
                                             columns=[NameSelector(pattern=text_column)],
                                             keys=[NameSelector(pattern=key_column)])
                             ])
                    ]),
                    DefaultExtractor(
                        ai_lab_config,
                        named_entity_recognition_enabled = False,# todo enable?
                        topic_classification_enabled = True,
                        keyword_search_enabled = False,
                        topics=topics,
                        parallelism_per_node=1)
                    ]
            ),
            output=Output(db_schema=schema, table_prefix="TEST_TXAI")
        )
        #self.pyexasol_con = conf.get(CKey.text_ai_pre_release_url)
        #self.temporary_db_object_schema = conf.get(CKey.temporary_db_object_schema)
        #self.language_alias = conf.get(CKey.language_alias)


    def run(self, conf: Secrets) -> None:
        activation_sql = get_activation_sql(conf)
        with open_pyexasol_connection(ai_lab_config, compression=True) as pyexasol_con:#todo or do these as inut?, or just call the run func directly and delete it here?
            #temporary_db_object_schema = conf.get(CKey.temporary_db_object_schema)
            pyexasol_con.execute(query=activation_sql)
            db_schema = conf.db_schema
            language_alias = "PYTHON3_TXAIE" #conf.get(CKey.language_alias)todo change once var exists in sectret store
            #self.run(pyexasol_con, temporary_db_object_schema, language_alias)
            super().run(pyexasol_con, db_schema, language_alias)
 



In [ ]:

BUCKETFS_CONNECTION_NAME=ai_lab_config.te_bfs_connection
BUCKETFS_SUB_DIR=ai_lab_config.te_models_bfs_dir
